In [7]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [5]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name = "Search Agent",
    tools = [WebSearchTool(search_context_size="low")],
    instructions=INSTRUCTIONS,
    model = "gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")  #By this be make it mandatory for the agent to run the specified tool
)

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**LangChain** is a widely adopted framework that enables developers to build applications powered by large language models (LLMs). It offers a modular design with extensive integrations, supporting multi-step workflows and complex agent loops. Key features include native support for various LLMs, a rich ecosystem of tools like Google Search and SQL, and capabilities for memory and conversation tracking. However, advanced workflows may require a solid understanding of prompt engineering and LLM behavior. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

**AutoGen**, developed by Microsoft, focuses on automating the generation of code, models, and processes for complex workflows. It leverages LLMs to assist developers in building, fine-tuning, and deploying AI solutions with minimal manual coding. AutoGen's architecture consists of three layers: Core, AgentChat, and Extensions, facilitating scalable and distributed networks of agents. It also provides tools like AutoGen Bench for performance assessment and AutoGen Studio for no-code agent development. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

**CrewAI** is designed for orchestrating role-playing AI agents in collaborative tasks, emphasizing simplicity and minimal setup. It allows for role-based agent assignments, simple implementation without complex dependencies, and collaborative workflows. CrewAI has gained popularity in customer service and marketing automation sectors. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

**LangGraph**, an extension of LangChain, introduces a graph-based system for managing data flow between agents. It's particularly useful for applications requiring decision loops or path changes based on new input, supporting branching, looping, and conditional flows. LangGraph is ideal for designing systems that mimic human decision-making in areas like loan processing and workflow automation. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

**Eliza** is an open-source, Web3-friendly AI agent operating system that integrates seamlessly with blockchain applications. It allows for reading and writing blockchain data, interacting with smart contracts, and is fully controlled by the user through regular TypeScript programs. Eliza aims to make deploying Web3 applications effortless. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**AutoAgent** is a fully automated, zero-code framework for LLM agents, enabling users to create and deploy agents through natural language alone. It comprises components like Agentic System Utilities, an LLM-powered Actionable Engine, a Self-Managing File System, and a Self-Play Agent Customization module. AutoAgent serves as a versatile multi-agent system for general AI assistants, demonstrating superior performance in generalist multi-agent tasks. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and approaches for building intelligent, autonomous systems. 

In [9]:
HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query")

    query: str = Field(description="The search term to use for the web search")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name = "PlannerAgent",
    instructions=INSTRUCTIONS,
    model = "gpt-4o-mini",
    output_type=WebSearchPlan
)

In [10]:
message = "Latest Agentic AI frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most recent developments and frameworks in agentic AI for 2025.', query='Agentic AI frameworks 2025'), WebSearchItem(reason='To gather insights on trends, updates, and new methodologies in agentic AI this year.', query='Latest trends in agentic AI 2025')]


In [11]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [12]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent", 
    model = "gpt-4o-mini",
    tools = [send_email],
    instructions=INSTRUCTIONS
)

In [13]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 2-3 pages of content, at least 500 words."
)

class ReportData(BaseModel):
    short_summary : str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report : str = Field(description="The final report")

    follow_up_questions : list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name = "Writer Agent",
    model = "gpt-4o-mini",
    instructions=INSTRUCTIONS,
    output_type=ReportData
)


The next 3 functions will plan and execute the search, using planner_agent and search_agent


In [14]:
async def plan_searches(query: str):
    """Use the planner agent to plan which searches to run for the query"""
    print("Planning Searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in Search Plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item) for item in search_plan.searches)]
    results = await asyncio.gather(*tasks)
    print("Finished Working")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in search plan"""
    input = f"Search term: {item.query}\n Reason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output